In [1]:
#%% 
import xarray as xr
from PyCO2SYS import sys
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd
import xarray as xr
import pandas as pd
from PyCO2SYS import sys
import warnings
warnings.filterwarnings("ignore")
import warnings

# 3rd party libraries

import scipy
import glob

In [2]:
#%%
yr_min = 1993
time_min = np.datetime64('1993-01-01')
time_max = np.datetime64('2021-12-31')
lon_min = -170
lon_max = -40
lat_min = 10
lat_max = 65
#%%
def calc_clim_anom(ts):
    ts_reshape = np.reshape(ts,[np.int64(len(ts)/12),12])
    ts_detrend = scipy.signal.detrend(ts,axis=0)
    ts_detrend_reshape = np.reshape(ts_detrend,ts_reshape.shape)
    clim = np.nanmean(ts_detrend_reshape,axis=0)
    ts_anom = ts_reshape - clim
    ts_anom = np.reshape(ts_anom,ts.shape)
    return clim,ts_anom
n_yr = np.int64(2022-yr_min)

In [3]:
file_socat = '/Volumes/Crucial_4T/carbon/SOCAT/v2023/SOCATv2023_qrtrdeg_gridded_coast_monthly.nc'
ds_socat = xr.open_dataset(file_socat)
lon_socat = ds_socat.xlon.values
lat_socat = ds_socat.ylat.values
time_socat = ds_socat.tmnth.values
idxx = np.where((lon_socat>=lon_min) & (lon_socat<=lon_max))[0]
idxy = np.where((lat_socat>=lat_min) & (lat_socat<=lat_max))[0]
idxt = np.where((time_socat>=time_min) & (time_socat<=time_max))[0]
lon_socat = lon_socat[idxx]
lat_socat = lat_socat[idxy]
time_socat = time_socat[idxt]
fco2_socat = ds_socat.coast_fco2_ave_weighted[idxt,idxy,idxx].values
sst_socat = ds_socat.coast_sst_ave_weighted[idxt,idxy,idxx].values
sss_socat = ds_socat.coast_salinity_ave_weighted[idxt,idxy,idxx].values
fco2_socat[(fco2_socat>1000)|(fco2_socat<0)] = np.nan
sst_socat[(sst_socat>100)|(sst_socat<-4)] = np.nan
sss_socat[(sss_socat>100)|(sss_socat<0)] = np.nan
# results = sys(par1=1000,par1_type=1,par2=fco2_socat,par2_type=5,temperature=sst_socat,temperature_out=sst_socat,salinity=sss_socat)
pco2_socat = fco2_socat*(1.00436 - 4.669 * 10**-5 * sst_socat)
pco2_socat[(pco2_socat>1000)|(pco2_socat<1)] = np.nan
pco2_socat[pco2_socat<=np.nanmean(pco2_socat)-3*np.nanstd(pco2_socat)] = np.nan
pco2_socat[pco2_socat>=np.nanmean(pco2_socat)+3*np.nanstd(pco2_socat)] = np.nan
pco2_socat_4d = np.reshape(pco2_socat,(np.int64(pco2_socat.shape[0]/12),12,pco2_socat.shape[1],pco2_socat.shape[2]))
pco2_socat_clim = np.nanmean(np.reshape(pco2_socat,(np.int64(pco2_socat.shape[0]/12),12,pco2_socat.shape[1],pco2_socat.shape[2])),axis=0)
sst_socat_4d = np.reshape(sst_socat,(np.int64(sst_socat.shape[0]/12),12,sst_socat.shape[1],sst_socat.shape[2]))
sss_socat_4d = np.reshape(sss_socat,(np.int64(sss_socat.shape[0]/12),12,sss_socat.shape[1],sss_socat.shape[2]))
fco2_socat_4d = np.reshape(fco2_socat,(np.int64(pco2_socat.shape[0]/12),12,pco2_socat.shape[1],pco2_socat.shape[2]))

In [4]:
# Read SSH data
file_i = '/Volumes/Crucial_4T/SSH/SEALEVEL_GLO_PHY_L4_MY_008_047/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D/2000/01/dt_global_allsat_phy_l4_20000101_20210726.nc'
ds_i = xr.open_dataset(file_i)
lon_ssh = ds_i.longitude.values
lat_ssh = ds_i.latitude.values
idxlon = (lon_ssh>=lon_min) & (lon_ssh<=lon_max)
idxlat = (lat_ssh>=lat_min) & (lat_ssh<=lat_max)
lon_ssh = lon_ssh[idxlon]
lat_ssh = lat_ssh[idxlat]

yr_start = yr_min
yr_end = 2021
yrs_ssh = np.arange(yr_start,yr_end+1)
adt_all = np.full((len(yrs_ssh)*12,len(lat_ssh),len(lon_ssh)),np.nan)
time_ssh_all = np.full(adt_all.shape[0],np.datetime64('1993-01-15'))
for idxyr in range(yr_start,yr_end+1):
    for idxmon in range(1,13):
        idxt = (idxyr-yr_start)*12+idxmon-1
        files = glob.glob('/Volumes/Crucial_4T/SSH/SEALEVEL_GLO_PHY_L4_MY_008_047/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D/'+str(idxyr)+'/'+str(idxmon).zfill(2)+'/*.nc')
        adt_i = np.full((len(files),len(lat_ssh),len(lon_ssh)),np.nan)
        for idxf in range(len(files)):
            file_i = files[idxf]
            ds_i = xr.open_dataset(file_i)
            adt_i[idxf] = ds_i.adt[:,idxlat,idxlon].values
        time_ssh_all[idxt] = np.datetime64(str(idxyr)+'-'+str(idxmon).zfill(2)+'-15')
        adt_all[idxt] = np.nanmean(adt_i,axis=0).copy()
adt_4d = adt_all.reshape(fco2_socat_4d.shape) 

In [5]:
#%%
file_soda = '/Volumes/Crucial_4T/Model/soda/3.15.2/soda3.15.2_mn_ocean_reg_'+str(yr_min)+'.nc'
ds_soda = xr.open_dataset(file_soda)
lon_soda = ds_soda.xt_ocean.values
lat_soda = ds_soda.yt_ocean.values
sss_4d = np.full((n_yr,12,len(lat_socat),len(lon_socat)),np.nan)

for yr in range(yr_min,2022):
    file_soda = '/Volumes/Crucial_4T/Model/soda/3.15.2/soda3.15.2_mn_ocean_reg_'+str(yr)+'.nc'
    ds_soda = xr.open_dataset(file_soda)
    sss_4d[yr-yr_min] = ds_soda.salt[:,0,:,:].interp(yt_ocean=lat_socat,xt_ocean=lon_socat+360).values

In [6]:
#%% SST
file_sst = '/Volumes/Crucial_4T/SST/OISST_v2.1/oisst-avhrr-v02r01.19810901.nc'
ds_sst = xr.open_dataset(file_sst)
lon_sst = ds_sst.lon.values
lat_sst = ds_sst.lat.values
idxlon = np.where((lon_sst>=lon_min+360)&(lon_sst<=lon_max+360))[0]
idxlat = np.where((lat_sst>=lat_min)&(lat_sst<=lat_max))[0]
sst_4d = np.full((n_yr,12,len(lat_socat),len(lon_socat)),np.nan)
for yr in range(yr_min,2022):
    for mon in range(1,13):
        idxt = np.int64((yr-yr_min)*12+mon-1)
        files_sst = glob.glob('/Volumes/Crucial_4T/SST/OISST_v2.1/oisst-avhrr-v02r01.'+str(yr)+str(mon).zfill(2)+'*.nc')
        sst_i = np.full((len(files_sst),len(idxlat),len(idxlon)),np.nan)
        for idxf in range(len(files_sst)):
            file_sst = files_sst[idxf]
            ds_sst = xr.open_dataset(file_sst)
            sst_i[idxf] = ds_sst.sst[:,:,idxlat,idxlon].values
        sst_4d[yr-yr_min,mon-1] = np.nanmean(sst_i,axis=0)

In [7]:
dsv2 = xr.open_dataset('/Volumes/T7_4T/CCMP/v02.0/Y1990/M01/CCMP_Wind_Analysis_199001_V02.0_L3.5_RSS.nc')
idxlatv2 = (dsv2.latitude.values>=lat_min) & (dsv2.latitude.values<=lat_max)
idxlonv2 = (dsv2.longitude.values>=lon_min+360) & (dsv2.longitude.values<=lon_max+360)

dsv3 = xr.open_dataset('/Volumes/T7_4T/CCMP/v03.1/Y1993/M01/CCMP_Wind_Analysis_199301_monthly_mean_V03.1_L4.0.nc')
idxlatv3 = (dsv3.latitude.values>=lat_min) & (dsv3.latitude.values<=lat_max)
idxlonv3 = (dsv3.longitude.values>=lon_min+360) & (dsv3.longitude.values<=lon_max+360)

sst_socat_4d = sst_socat.reshape(pco2_socat_4d.shape)
sss_socat_4d = sss_socat.reshape(pco2_socat_4d.shape)

dsv2 = xr.open_dataset('/Volumes/T7_4T/CCMP/v02.0/Y1990/M01/CCMP_Wind_Analysis_199001_V02.0_L3.5_RSS.nc')
idxlatv2 = (dsv2.latitude.values>=lat_min) & (dsv2.latitude.values<=lat_max)
idxlonv2 = (dsv2.longitude.values>=lon_min+360) & (dsv2.longitude.values<=lon_max+360)

dsv3 = xr.open_dataset('/Volumes/T7_4T/CCMP/v03.1/Y1993/M01/CCMP_Wind_Analysis_199301_monthly_mean_V03.1_L4.0.nc')
idxlatv3 = (dsv3.latitude.values>=lat_min) & (dsv3.latitude.values<=lat_max)
idxlonv3 = (dsv3.longitude.values>=lon_min+360) & (dsv3.longitude.values<=lon_max+360)

wspd_4d = np.zeros(sst_4d.shape)
for yr in range(yr_min,2021):
    for mon in range(1,13):
        if yr < 1993:
            dsv2 = xr.open_dataset('/Volumes/T7_4T/CCMP/v02.0/Y'+str(yr)+'/M'+str(mon).zfill(2)+'/CCMP_Wind_Analysis_'+str(yr)+str(mon).zfill(2)+'_V02.0_L3.5_RSS.nc')
            wspd_4d[yr-yr_min,mon-1] = dsv2.wspd[:,idxlatv2,idxlonv2].values
        else:
            dsv3 = xr.open_dataset('/Volumes/T7_4T/CCMP/v03.1/Y'+str(yr)+'/M'+str(mon).zfill(2)+'/CCMP_Wind_Analysis_'+str(yr)+str(mon).zfill(2)+'_monthly_mean_V03.1_L4.0.nc')
            wspd_4d[yr-yr_min,mon-1] = dsv3.w[idxlatv3,idxlonv3].values

In [8]:
file_air = '/Volumes/Crucial_4T/carbon/NOAA_MBL_Reference/Global_90S90N_Surface.txt'
df_air = pd.read_csv(file_air,header=81,sep='\s+')
xCO2_data = df_air.iloc[:,1:-1:2].values
xCO2_err = df_air.iloc[:,2::2].values
time_xCO2 = df_air.iloc[:,0].values
lat_xCO2 = np.rad2deg(np.arcsin(np.array([-1.00,-0.95,-0.90,-0.85,-0.80,-0.75,-0.70,-0.65,-0.60,-0.55,-0.50,-0.45,-0.40,-0.35,-0.30,-0.25,-0.20,-0.15,-0.10,-0.05,0.00,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1.00])))
ds_xCO2air = xr.Dataset(
    data_vars=dict(
        xCO2air=(['time','lat'], xCO2_data),
        xCO2air_err=(['time','lat'], xCO2_err),
        ),
    coords=dict(
        time=time_xCO2,
        lat=lat_xCO2,
        )
)
xCO2air = ds_xCO2air.xCO2air.interp(time=np.arange(yr_min+1/24,2022,1/12),lat=lat_socat).values
xCO2air = np.repeat(np.reshape(xCO2air,(xCO2air.shape[0],xCO2air.shape[1],1)),axis=2,repeats=sst_4d.shape[-1])
xCO2air = np.reshape(xCO2air,sss_4d.shape)
results = sys(par1=xCO2air,par1_type=9,temperature=sst_4d,temperature_out=sst_4d,salinity=sss_4d,pressure=0,pressure_out=0)
# pco2air = co_xco2topco2(xCO2air, sst, sss)
pco2air = results['pCO2_out']
pco2air[pco2air<200] = np.nan

In [9]:
file_i = '/Volumes/Crucial_4T/SSH/SEALEVEL_GLO_PHY_L4_MY_008_047/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1M-m/1993/dt_global_allsat_msla_h_y1993_m01.nc'
ds_i = xr.open_dataset(file_i)
lon_ssh = ds_i.longitude.values
lat_ssh = ds_i.latitude.values
idxlon = np.where((lon_ssh>=lon_min+360)&(lon_ssh<=lon_max+360))[0]
idxlat = np.where((lat_ssh>=lat_min)&(lat_ssh<=lat_max))[0]

sla_4d = np.full((n_yr,12,len(idxlat),len(idxlon)),np.nan)

for yr in range(yr_min,2022):
    for m in range(1,13):
        file_i = '/Volumes/Crucial_4T/SSH/SEALEVEL_GLO_PHY_L4_MY_008_047/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1M-m/'+str(yr)+'/dt_global_allsat_msla_h_y'+str(yr)+'_m'+str(m).zfill(2)+'.nc'
        ds_i = xr.open_dataset(file_i)
        sla_4d[yr-1993,m-1] = np.squeeze(ds_i.sla[:,idxlat,idxlon].values).copy()

In [10]:
# Read CHLA data
from datetime import datetime, timedelta
files_chla = glob.glob('/Volumes/T7_4T/modis/mapped_monthly/chlor_a/*.nc')
ds_chla = xr.open_dataset(files_chla[0])
lon_chla = ds_chla.lon.values
lat_chla = ds_chla.lat.values
idxlon = (lon_chla>=lon_min) & (lon_chla<=lon_max)
idxlat = (lat_chla>=lat_min) & (lat_chla<=lat_max)
chla_i = ds_chla.chlor_a[idxlat,idxlon].interp(lat=lat_socat,lon=lon_socat,method='linear').values
chla = np.full((n_yr,12,len(lat_socat),len(lon_socat)),np.nan)
for file in files_chla:
    ds_chla = xr.open_dataset(file)
    chla_i = ds_chla.chlor_a[idxlat,idxlon].interp(lat=lat_socat,lon=lon_socat,method='linear').values
    filename = file[44:]

    # Extract the year and day of year
    year = int(filename[1:5])
    day_of_year = int(filename[5:8])

    # Convert day of the year to date
    date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)  # subtract 1 because Jan 1 is day 1
    idxyr = year - yr_min
    idxmon = date.month - 1
    chla[idxyr,idxmon] = chla_i.copy()

In [11]:
# file = '/Volumes/Crucial_4T/carbon/SOM_FNN/ULB-SOM-FFN-coastalv2.nc'
# ds_somfnn = xr.open_dataset(file)
# fco2_product = ds_somfnn.pco2.interp(lon=lon_socat,lat=lat_socat,time=time_socat).values

In [12]:
# lat_msh, lon_msh = np.meshgrid(lat_socat,lon_socat,indexing='ij')
# file_topo = '/Volumes/Crucial_4T/etopo/ETOPO2/ETOPO2v2c_f4.nc'
# ds_topo = xr.open_dataset(file_topo)
# depth_wide = ds_topo.z.interp(x=lon_socat,y=lat_socat).values

# mask_wide = dict({})

# mask_wide['mask_GoMeSS'] = np.float64((lon_msh>-70)&(lon_msh<=-60)&(lat_msh>=41.5)&(lat_msh<=46))
# mask_wide['mask_GoMeSS'][mask_wide['mask_GoMeSS']<0.5] = np.nan
# mask_wide['mask_GoMeSS'] = np.repeat(np.reshape(mask_wide['mask_GoMeSS'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_GoMeSS'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_GoMe'] = np.float64((lon_msh>-71)&(lon_msh<=-65.5)&(lat_msh>=30)&(lat_msh<=46))
# mask_wide['mask_GoMe'][mask_wide['mask_GoMe']<0.5] = np.nan
# mask_wide['mask_GoMe'] = np.repeat(np.reshape(mask_wide['mask_GoMe'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_GoMe'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_SS'] = np.float64((lon_msh>-65.5)&(lon_msh<=-60)&(lat_msh>=37)&(lat_msh<=46))
# mask_wide['mask_SS'][mask_wide['mask_SS']<0.5] = np.nan
# mask_wide['mask_SS'] = np.repeat(np.reshape(mask_wide['mask_SS'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_SS'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_GStL'] = np.float64((lon_msh>-69)&(lon_msh<=-45)&(lat_msh>=41.5)&(lat_msh<=51))
# mask_wide['mask_GStL'][mask_wide['mask_GStL']<0.5] = np.nan
# mask_wide['mask_GStL'] = np.repeat(np.reshape(mask_wide['mask_GStL'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_GStL'][mask_wide['mask_GoMeSS']>0.5] = np.nan
# mask_wide['mask_GStL'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_SAB'] = np.float64((lon_msh>=-82)&(lon_msh<=-70)&(lat_msh>=27)&(lat_msh<=35.5))
# mask_wide['mask_SAB'][mask_wide['mask_SAB']<0.5] = np.nan
# mask_wide['mask_SAB'] = np.repeat(np.reshape(mask_wide['mask_SAB'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_SAB'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_MAB'] = np.float64((lon_msh>=-82)&(lon_msh<=-70)&(lat_msh>=35.5)&(lat_msh<=41.5))
# mask_wide['mask_MAB'][mask_wide['mask_MAB']<0.5] = np.nan
# mask_wide['mask_MAB'] = np.repeat(np.reshape(mask_wide['mask_MAB'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_MAB'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_GoMx'] = np.float64((lon_msh>=-100)&(lon_msh<=-80)&(lat_msh>=17)&(lat_msh<=31))
# mask_wide['mask_GoMx'][mask_wide['mask_GoMx']<0.5] = np.nan
# x1 = -83; y1 = 31; x2 = -81; y2 =25.5
# a = (y2-y1)/(x2-x1)
# b = y2-a*x2
# mask_wide['mask_GoMx'][(a*lon_msh+b)<lat_msh] = np.nan
# x1 = -87.0; y1 = 21; x2 = -80.5; y2 =25
# a = (y2-y1)/(x2-x1)
# b = y2-a*x2
# mask_wide['mask_GoMx'][(a*lon_msh+b)>lat_msh] = np.nan
# mask_wide['mask_GoMx'][(lon_msh<=-89.75) & (lat_msh>=29.5)] = np.nan
# mask_wide['mask_GoMx'] = np.repeat(np.reshape(mask_wide['mask_GoMx'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_GoMx'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_CbS'] = np.float64((lon_msh>=-90)&(lon_msh<=-55)&(lat_msh>=15)&(lat_msh<=27))
# mask_wide['mask_CbS'][mask_wide['mask_CbS']<0.5] = np.nan
# mask_wide['mask_CbS'] = np.repeat(np.reshape(mask_wide['mask_CbS'],(1,len(lat_socat),len(lon_socat))),len(time_socat),axis=0)
# mask_wide['mask_CbS'][~np.isnan(mask_wide['mask_GoMx'])] = np.nan
# mask_wide['mask_CbS'][~np.isnan(mask_wide['mask_SAB'])] = np.nan
# mask_wide['mask_CbS'][np.isnan(fco2_product)] = np.nan

# mask_wide['mask_Atlantic'] = np.float64((~np.isnan(mask_wide['mask_GoMx']) + ~np.isnan(mask_wide['mask_SAB']) + ~np.isnan(mask_wide['mask_MAB']) + ~np.isnan(mask_wide['mask_GoMe']) + ~np.isnan(mask_wide['mask_SS']) + ~np.isnan(mask_wide['mask_GStL']))>0)
# mask_wide['mask_Atlantic'][mask_wide['mask_Atlantic']<0.5] = np.nan
# mask_wide['mask_Atlantic'][np.isnan(fco2_product)] = np.nan


In [13]:

# fco2_socat_4d = fco2_socat_4d * mask_wide['mask_Atlantic'][0]
# sss_4d = sss_4d * mask_wide['mask_Atlantic'][0]
# sst_4d = sst_4d * mask_wide['mask_Atlantic'][0]
# sla_4d = sla_4d * mask_wide['mask_Atlantic'][0]
# adt_4d = adt_4d * mask_wide['mask_Atlantic'][0]
# wspd_4d = wspd_4d * mask_wide['mask_Atlantic'][0]
# pco2air_4d = pco2air * mask_wide['mask_Atlantic'][0]
# chla_4d = chla * mask_wide['mask_Atlantic'][0]


In [14]:
ds = xr.Dataset(
    data_vars=dict(
        fco2_socat =(["year", "month", "lat", "lon"], fco2_socat_4d),
        sss =(["year", "month", "lat", "lon"], sss_4d),
        sst =(["year", "month", "lat", "lon"], sst_4d),
        sla =(["year", "month", "lat", "lon"], sla_4d),
        adt =(["year", "month", "lat", "lon"], adt_4d),
        wspd =(["year", "month", "lat", "lon"], wspd_4d),
        pco2air =(["year", "month", "lat", "lon"], pco2air),
        xco2air = (["year", "month", "lat", "lon"], xCO2air),
        chla =(["year", "month", "lat", "lon"], chla),
    ),
    coords={
        "year": np.arange(1993,2022),
        "month": np.arange(1,13),
        "lat": lat_socat,
        "lon": lon_socat,
    },
)
ds.to_netcdf("data/satellite_socat_NACCOM.nc")

: 